In [251]:
import pandas as pd
import numpy as np
import warnings

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    # path = '~/Documents/GitHub/f1-analytics/'
    path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [252]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/merged.csv')

In [253]:
### Season to test results

N = 2016

In [254]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season != N]

scaler = StandardScaler()
X_train = train.drop(['podium'], axis=1)
# X_train = train.drop(['podium','date', 'fastest_lap', 'season', 'round', 'pos'], axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
# X_train = np.asarray(X_train)

y_train = np.asarray(train.podium.values)

In [255]:
def score_classification(model):
    probabilities = []
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(['podium'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns=['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df.reset_index(inplace=True, drop=True)

        probabilities += list(prediction_df['proba_1'].values)

    return probabilities

In [256]:
svm_wins = svm.SVC(probability=True, kernel="linear",gamma='auto')   # Initialize the SVM model

svm_wins.fit(X_train, y_train)                                      # Train the SVM model

SVC(gamma='auto', kernel='linear', probability=True)

In [257]:
probabilities = score_classification(svm_wins)

test = merged[(merged.season == N)]

test['prob'] = probabilities
test = test[['round', 'driver', 'prob', 'grid', 'podium']]
max_prob = test.groupby('round').agg({'prob': max}).reset_index()

predict_wins = pd.merge(test, max_prob, how='inner', on=['round', 'prob']).drop('prob', axis=1)

predict_wins

,round,driver,grid,podium
0,1,hamilton,1,2
1,2,hamilton,1,3
2,3,rosberg,1,1
3,4,rosberg,1,1
4,5,max_verstappen,4,1
5,6,rosberg,2,7
6,7,hamilton,1,1
7,8,rosberg,1,1
8,9,hamilton,1,1
9,10,hamilton,1,1
